In [127]:
# BLOCK 1: IMPORTS

# Python Library Imports 
import pandas as pd  # for Dataframes
import numpy as np  # for arrays
import matplotlib.pyplot as plt  # for plotting visuals
# import operator

# import xgboost as xgb  # boosted decision trees
# import pandas_ml as pdml

# import sklearn  # for machine learning
# from sklearn import tree 
# from sklearn import preprocessing

# from sklearn.preprocessing import scale
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler  
# from sklearn.metrics import log_loss, accuracy_score
# from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, export_graphviz
# from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

# from IPython.display import Image

# import pickle # persistant objects aka classifier after training

# from collections import Counter
# from sklearn.datasets import make_classification
# import subprocess

# from sklearn.datasets import load_svmlight_files
# from sklearn.metrics import accuracy_score
# from xgboost.sklearn import XGBClassifier, XGBRegressor
# from sklearn.metrics import mean_squared_error

In [3]:
# # BLOCK 2: STYLING And Globals

# # Styling
# pd.set_option('notebook_repr_html', True)
# pd.set_option('max_columns', 50)
# # pd.set_option('display.width', 1000)
# pd.set_option('max_colwidth', 40)
# %matplotlib inline 
# # pd.set_option('display.max_columns', None)

In [253]:
import pandas as pd  # for Dataframes
import numpy as np  # for arrays
import matplotlib.pyplot as plt  # for plotting visuals

class Preprocessing():
    def __init__(self):
        self.df = -1 #main dataframe
        
        self.log = [] #log of everything list
        self.logCount = 0 #what number log count for debugging
    
    # Imports file into a dataframe:df for class
    def dataImport(self, filename, filetype):
        s = "LOG:" + str(self.logCount)
        if (type(self.df) == pd.DataFrame):
            s += " You have already imported FILE"
        else:
            s += " Importing file into a -CLASS Dataframe"
            try:
                self.df = pd.read_csv(filename, index_col=0)
            except:
                s += " -INPUT FILE EXCEPTION"
                
        self.log.append(s)
        self.logCount += 1
        return(self.log, self.df)
    
    # 1 Categorical function that returns a list of all the category columns
    def categoryFeatures(self, *file):
        s = "LOG:" + str(self.logCount)
        catColList = []
        for cat in range(self.df.shape[1]):
            if self.df.dtypes[cat] == 'object':
                catColList.append(self.df.columns[cat])
                
        s += " CategoryFeatures: " + str(catColList)
        self.log.append(s)
        self.logCount += 1
        return(self.log, self.df, catColList)
    
    # Categorical function that returns all the counts for each unique category within the category column
    def categoryCounts(self, df, catColList, *cmpCols):
        s = "LOG:" + str(self.logCount)
                        
        categTotals = [] # list to hold the totals for each unique category with each category column

        cmpColLists = [] # a list of lists for the additional comparison columns
        for each in cmpCols: # create a list for each additional parameter
            cmpColLists.append([])
        
        for catIndex, cat in enumerate(catColList): # for each category out of the columns
            # the general idea is to create a dictionary for each unique, get info, then append it to either categTotals or cmpColLists
            categTotalsd = {}
            
            others = []
            for each in cmpCols:
                others.append({})
            
            # For each unique category in the column
            # catUniqueIndex = index of each unique category
            # uniqueCat = a category within that category column
            # e.g 
            # Category Column, roll_up, has Unique Categories 'retention', 'unmanaged', 'onboarding'
            for catUniqueIndex, uniqueCat in enumerate(df[cat].unique()):
                categTotalsd[uniqueCat] = df[cat].value_counts()[uniqueCat]  # totals of each uniqueCategory in category columns
                
                # for each in the list of comparison columns
                for i, each in enumerate(cmpCols):
                    tempDfBool = (df[each]>0) & (df[cat] == uniqueCat) # boolean df of all the positives
                    tempDfCats = df[tempDfBool] # df of just positives 
                    others[i][uniqueCat] = tempDfCats[each].sum().round(2) # sum of the column row
            
            #appending for category totals
            categTotals.append(cat)
            categTotals.append(categTotalsd)    
            
            #appending for each in list of comparison columns
            for i, each in enumerate(cmpCols): 
                cmpColLists[i].append(cat)
                cmpColLists[i].append(others[i])
                
        s += " Found counts for: " + str(catColList) + "\n"
        s += "Found uniques and their counts too"
        self.log.append(s)
        self.logCount += 1
        return self.log, self.df, categTotals, cmpColLists
    
    def categoryStats(self, df, categTotals, cmpColLists):
        s = "LOG:" + str(self.logCount)
        assert(len(cmpColLists) >= 2)
        
        pcntPos = []
        avgPrice = []
        
        for totals, item, item2 in zip(categTotals, cmpColLists[0], cmpColLists[1]):
            pcntPosd = {}
            avgPriced = {}
            if type(item) == dict and type(item2) == dict:
                for x in item:
                    pcntPosd[x] = item.get(x) / totals.get(x)
                    if(item.get(x) == 0):
                        avgPriced[x] = 0
                    else:
                        avgPriced[x] = item2.get(x) / item.get(x)
                pcntPos.append(pcntPosd)
                avgPrice.append(avgPriced)
                
        s += " categoryStats function: %positive for each cat, avgPrice of buy"
        self.log.append(s)
        self.logCount += 1        
        return self.log, self.df, pcntPos, avgPrice
    
    def meanDf(self, df, classification, regression):
        totalInDf = len(df)
        
        classi = df[classification] > 0
        classi = df[classi]
        totalClass = len(classi)

        regre = df[regression] > 0
        regre = df[regre]
        totalRegre = len(regre)

        
    
    
def lPrint(objList):
    for i, each in enumerate(objList):
        print(each)

pre = Preprocessing()
log, df = pre.dataImport('files/small.csv','csv')
log, df, catColList = pre.categoryFeatures()
log, df, categTotals, cmpColLists = pre.categoryCounts(df, catColList, 'convert_30', 'revenue_30')
log, df, pcntPos, avgPrice = pre.categoryStats(df, categTotals, cmpColLists)
pre.meanDf(df, 'convert_30', 'revenue_30')

# lPrint(pcntPos)
# lPrint( avgPrice)
# print(log)
# print(cmpColLists)


# listPrinter(cmpColLists[1])



In [138]:
class Printer():
    def __init__():
        pass

In [87]:
def f(): 
    global s 
    print('1', s) 
    s = "Look for Geeksforgeeks Python Section"
    print(s)  
  
# Global Scope 
s = "Python is great!" 
f() 
print(s)

1 Python is great!
Look for Geeksforgeeks Python Section
Look for Geeksforgeeks Python Section


In [22]:
  # append to lists     
#             categTotalsd = sorted(categTotalsd.items(), key=operator.itemgetter(1), reverse=True)